In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
from tqdm import tqdm
from torchvision.io import read_image
import pandas as pd
import numpy as np

# Select OI labels to download 

In [2]:
labels_path = "/bigstor/zsarwar/OpenImages/oidv7-train-annotations-human-imagelabels.csv"
image_save_path = "/bigstor/zsarwar/OpenImages/train/"
label_mapping_file = "/bigstor/zsarwar/OpenImages/oidv7-class-descriptions.csv"
select_images_path_txt_path = "/bigstor/zsarwar/OpenImages/selected_training_images_dogs.txt"
#select_images_path_images_path = "/bigstor/zsarwar/OpenImages/Images.txt"
df_save_path = "/bigstor/zsarwar/OpenImages/"

In [3]:
# Desired classes from Imagenet
#desired_imagenet_classes = ['Siberian husky','Australian terrier','bloodhound, sleuthhound','black-and-tan coonhound','miniature poodle','Yorkshire terrier','toy terrier','Appenzeller','collie','Eskimo dog, husky','English foxhound','briard']

#desired_imagenet_classes = ['Siberian husky', 'Australian terrier', 'malamute, malemute, Alaskan malamute', 'Great Dane', 'Walker hound, Walker foxhound', 'soft-coated wheaten terrier', 'otterhound, otter hound', 'bloodhound, sleuthhound', 'black-and-tan coonhound', 'Staffordshire bullterrier, Staffordshire bull terrier', 'miniature poodle', 'Cardigan, Cardigan Welsh corgi', 'Lhasa, Lhasa apso', 'Yorkshire terrier', 'Irish wolfhound', 'EntleBucher', 'Norfolk terrier', 'toy terrier', 'toy poodle', 'Scotch terrier, Scottish terrier, Scottie', 'Doberman, Doberman pinscher', 'Appenzeller', 'Irish water spaniel', 'miniature schnauzer', 'collie', 'Border collie', 'boxer', 'silky terrier, Sydney silky', 'Norwich terrier', 'flat-coated retriever', 'Eskimo dog, husky', 'Lakeland terrier', 'Tibetan terrier, chrysanthemum dog', 'wire-haired fox terrier', 'basset, basset hound', 'American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier', 'Chihuahua', 'redbone', 'Sussex spaniel', 'Rhodesian ridgeback', 'Ibizan hound, Ibizan Podenco', 'English foxhound', 'briard']

#corr_oi_classes = ["Miniature siberian husky", "Siberian husky", "Australian terrier",                              "Bloodhound", "Black and tan coonhound", "Miniature Poodle", "Yorkshire terrier", "English toy terrier", "Appenzeller sennenhund", "Border collie", "Collie", "Australian collie","Canadian eskimo dog", "English foxhound", "Briard"]

#corr_oi_classes = ["Backpack", "Strawberries", "Strawberry","Virginia strawberry", "Alpine strawberry", "Mandarin orange", "Orange (fruit)", "Valencia orange", "Bitter orange", "Blood orange", "Sweet lemon",
#                    "Lemon peel", "Meyer lemon", "Bitter lemon"]

In [4]:
imagenet_oi_map = {
'English setter': ["English setter"],
 'Siberian husky': ['Miniature siberian husky', 'Siberian husky'],
 'Australian terrier': ['Australian terrier'],
 'malamute, malemute, Alaskan malamute': ["Alaskan malamute"],
 'Great Dane': ["Great dane"],
 'Walker hound, Walker foxhound': ["Treeing walker coonhound"],
 'whippet': ["Whippet"],
 'Scottish deerhound, deerhound': ["Scottish deerhound"],
 'soft-coated wheaten terrier': ["Irish soft-coated wheaten terrier"],
 'Dandie Dinmont, Dandie Dinmont terrier': ["Dandie dinmont terrier"],
 'otterhound, otter hound': ["Otterhound"],
 'bloodhound, sleuthhound': ['Bloodhound'],
 'Airedale, Airedale terrier': ["Airedale terrier"],
 'giant schnauzer': ["Giant schnauzer"],
 'black-and-tan coonhound': ['Black and tan coonhound'],
 'Staffordshire bullterrier, Staffordshire bull terrier': ["Staffordshire bull terrier"],
 'Bouvier des Flandres, Bouviers des Flandres': ["Bouvier des flandres"],
 'miniature poodle': ['Miniature Poodle'],
 'Cardigan, Cardigan Welsh corgi': ["Cardigan welsh corgi"],
 'Lhasa, Lhasa apso': ["Lhasa apso"],
 'Yorkshire terrier': ['Yorkshire terrier'],
 'Saluki, gazelle hound': ["Saluki"],
 'Sealyham terrier, Sealyham': ["Sealyham terrier"],
 'Irish wolfhound': ["Irish wolfhound"],
 'EntleBucher': ["Entlebucher mountain dog"],
 'Maltese dog, Maltese terrier, Maltese': ["Maltese"],
 'Norfolk terrier': ["Norfolk terrier"],
 'toy terrier': ['English toy terrier'],
 'vizsla, Hungarian pointer': ["Vizsla"],
 'cairn, cairn terrier': ["Cairn terrier"],
 'groenendael': ["Groenendael"],
 'Japanese spaniel': ["Japanese chin"],
 'toy poodle': ["Toy Poodle"],
 'Kerry blue terrier': ["Kerry blue terrier"],
 'Scotch terrier, Scottish terrier, Scottie': ["Scottish terrier"],
 'Doberman, Doberman pinscher': ["Dobermann"],
 'Greater Swiss Mountain dog': ["Greater swiss mountain dog"],
 'Appenzeller': ['Appenzeller sennenhund'],
 'Shih-Tzu': ["Shih tzu"],
 'Irish water spaniel': ["Irish water spaniel"],
 'miniature schnauzer': ["Miniature schnauzer"],
 'collie': ['Border collie', 'Collie', 'Australian collie'],
 'Irish terrier': ["Irish terrier"],
 'affenpinscher, monkey pinscher, monkey dog': ["Affenpinscher"],
 'Border collie': ["Border collie"],
 'boxer': ["Boxer"],
 'silky terrier, Sydney silky': ["Australian silky terrier"],
 'beagle': ["Beagle"],
 'German short-haired pointer': ["German shorthaired pointer"],
 'Chesapeake Bay retriever': ["Chesapeake bay retriever"],
 'kuvasz': ["Kuvasz"],
 'curly-coated retriever': ["Murray river curly coated retriever"],
 'Norwich terrier': ["Norwich terrier"],
 'flat-coated retriever': ["Flat-coated retriever"],
 'keeshond': ["Keeshond"],
 'Eskimo dog, husky': ['Canadian eskimo dog'],
 'standard poodle': ["Standard Poodle"],
 'Lakeland terrier': ["Lakeland terrier"],
 'Gordon setter': ["Gordon setter"],
 'dingo, warrigal, warragal, Canis dingo': ["Dingo"],
 'Tibetan terrier, chrysanthemum dog': ["Tibetan terrier"],
 'basset, basset hound': ["Basset hound"],
 'American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier': ["American staffordshire terrier", "American pit bull terrier", "American staffordshire terrier"],
 'Shetland sheepdog, Shetland sheep dog, Shetland': ["Shetland sheepdog"],
 'Great Pyrenees': ["Great pyrenees"],
 'Chihuahua': ["Chihuahua"],
 'Labrador retriever': ["Labrador retriever"],
 'redbone': ["Redbone coonhound"],
 'kelpie': ["Australian kelpie"],
 'miniature pinscher': ["Miniature pinscher"],
 'Italian greyhound': ["Italian greyhound"],
 'cocker spaniel, English cocker spaniel, cocker': ["English cocker spaniel"],
 'Irish setter, red setter': ["Irish setter"],
 'Sussex spaniel': ["Sussex spaniel"],
 'Rhodesian ridgeback': ["Rhodesian ridgeback"],
 'Ibizan hound, Ibizan Podenco': ["Ibizan hound"],
 'English foxhound': ['English foxhound'],
 'briard': ['Briard'],
 'German shepherd, German shepherd dog, German police dog, alsatian': ["German shepherd"]}


In [5]:

"""
imagenet_oi_map = {'Siberian husky': ["Miniature siberian husky", "Siberian husky"],
                   'Australian terrier': ["Australian terrier"],
                   'bloodhound, sleuthhound': ["Bloodhound"],
                   'black-and-tan coonhound': ["Black and tan coonhound"],
                   'miniature poodle': ["Miniature Poodle"],
                   'Yorkshire terrier': ["Yorkshire terrier"],
                   'toy terrier': ["English toy terrier"],
                   'Appenzeller': ["Appenzeller sennenhund"],
                   'collie': ["Border collie", "Collie", "Australian collie"],
                   'Eskimo dog, husky': ["Canadian eskimo dog"],
                   'English foxhound': ["English foxhound"],
                   'briard': ["Briard"]}


imagenet_oi_map = {"backpack, back pack, knapsack, packsack, rucksack, haversack": ["Backpack"],
                "strawberry": ["Strawberries", "Strawberry","Virginia strawberry", "Alpine strawberry"],
                "orange" :["Mandarin orange", "Orange (fruit)", "Valencia orange", "Bitter orange", "Blood orange",] ,
                "lemon":["Sweet lemon","Lemon peel", "Meyer lemon", "Bitter lemon"]                  
                }
"""


'\nimagenet_oi_map = {\'Siberian husky\': ["Miniature siberian husky", "Siberian husky"],\n                   \'Australian terrier\': ["Australian terrier"],\n                   \'bloodhound, sleuthhound\': ["Bloodhound"],\n                   \'black-and-tan coonhound\': ["Black and tan coonhound"],\n                   \'miniature poodle\': ["Miniature Poodle"],\n                   \'Yorkshire terrier\': ["Yorkshire terrier"],\n                   \'toy terrier\': ["English toy terrier"],\n                   \'Appenzeller\': ["Appenzeller sennenhund"],\n                   \'collie\': ["Border collie", "Collie", "Australian collie"],\n                   \'Eskimo dog, husky\': ["Canadian eskimo dog"],\n                   \'English foxhound\': ["English foxhound"],\n                   \'briard\': ["Briard"]}\n\n\nimagenet_oi_map = {"backpack, back pack, knapsack, packsack, rucksack, haversack": ["Backpack"],\n                "strawberry": ["Strawberries", "Strawberry","Virginia strawberry"

In [6]:
desired_imagenet_classes = list(imagenet_oi_map.keys())
corr_oi_classes = list(imagenet_oi_map.values())

In [7]:
corr_classes = []
for i in range(len(corr_oi_classes)):
    for ele in corr_oi_classes[i]:
        corr_classes.append(ele)

corr_oi_classes = corr_classes

In [8]:
# First select the list of labels from label file
df_label_map = pd.read_csv(label_mapping_file)

df_label_ids = df_label_map[df_label_map['DisplayName'].isin(corr_oi_classes)]
label_ids = df_label_ids['LabelName'].tolist()
df_label_ids


,LabelName,DisplayName
175,/m/01p2dp,Affenpinscher
256,/m/017t3p,Airedale terrier
293,/m/01p3d7,Alaskan malamute
519,/m/049jr6,American pit bull terrier
526,/m/04m4bq,American staffordshire terrier
...,...,...
19102,/m/0xn5dj9,Toy Poodle
19341,/m/0773fb,Treeing walker coonhound
19958,/m/0183xd,Vizsla
20393,/m/08626,Whippet


In [9]:
df = pd.read_csv(labels_path)

In [10]:
len(df)

58783034

In [11]:
len(label_ids)

82

In [12]:
df_conf = df[df['Confidence'] == 1]
df_sub = df_conf[df_conf['LabelName'].isin(label_ids)]
image_id = df_sub['ImageID'].tolist()
image_id = ["train/" + id + "\n" for id in image_id]

dup_indexes = df_sub[df_sub['ImageID'].duplicated()].index
df_sub = df_sub.drop(dup_indexes)


In [10]:
# with open(select_images_path_txt_path, 'w') as o_file:
    #o_file.writelines(image_id)


# Run bash_downloader.sh

# Process Downloaded OpenImages 

In [13]:
# Remove images which never got downloaded
all_images = glob("/bigstor/zsarwar/OpenImages/train/*")
all_images_ids = [img.split("/")[-1].split(".")[0] for img in all_images]
df_sub = df_sub[df_sub['ImageID'].isin(all_images_ids)]

In [14]:
len(df_sub)

3756

In [15]:
# Load labels for the classifier you need to map OI labels to


In [16]:
df_imagenet_train = pd.read_pickle("/bigstor/zsarwar/Imagenet_2012_subsets/Imagenet_full/df_imagenet_dogs_train.pkl")

In [17]:
unique_classes = df_imagenet_train[df_imagenet_train['class'].isin(desired_imagenet_classes)]['class'].unique()

#df_imagenet_train.drop_duplicates(unique_classes)

In [18]:
unique_classes

array(['English setter', 'Siberian husky', 'Australian terrier',
       'malamute, malemute, Alaskan malamute', 'Great Dane',
       'Walker hound, Walker foxhound', 'whippet',
       'Scottish deerhound, deerhound', 'soft-coated wheaten terrier',
       'Dandie Dinmont, Dandie Dinmont terrier',
       'otterhound, otter hound', 'bloodhound, sleuthhound',
       'Airedale, Airedale terrier', 'giant schnauzer',
       'black-and-tan coonhound',
       'Staffordshire bullterrier, Staffordshire bull terrier',
       'Bouvier des Flandres, Bouviers des Flandres', 'miniature poodle',
       'Cardigan, Cardigan Welsh corgi', 'Lhasa, Lhasa apso',
       'Yorkshire terrier', 'Saluki, gazelle hound',
       'Sealyham terrier, Sealyham', 'Irish wolfhound', 'EntleBucher',
       'Maltese dog, Maltese terrier, Maltese', 'Norfolk terrier',
       'toy terrier', 'vizsla, Hungarian pointer', 'cairn, cairn terrier',
       'groenendael', 'Japanese spaniel', 'toy poodle',
       'Kerry blue terrier', '

In [19]:
duplicates = df_imagenet_train[df_imagenet_train.duplicated(subset='class')].index

In [20]:
df_imagenet_train = df_imagenet_train.drop(duplicates)

In [21]:
oi_imagenet_class = {}

for key in imagenet_oi_map.keys():
    for oi_class in imagenet_oi_map[key]:
        oi_imagenet_class[oi_class] = key


oi_labels_class = {}
for i in range(len(df_label_ids)):
    oi_labels_class[df_label_ids['LabelName'].iloc[i]] = df_label_ids['DisplayName'].iloc[i]


In [22]:

imagenet_class_id = {}
for i in range(len(df_imagenet_train)):
    imagenet_class_id[df_imagenet_train['class'].iloc[i]] = df_imagenet_train['label'].iloc[i]

oi_imagenet_label = {}
for lab in oi_labels_class.keys():
    oi_class = oi_labels_class[lab]
    imagenet_class = oi_imagenet_class[oi_class]
    imagenet_label = imagenet_class_id[imagenet_class]
    oi_imagenet_label[lab] = imagenet_label
    
imagenet_id_class = {v : k for k, v in imagenet_class_id.items()}    
oi_labels = df_sub['LabelName'].tolist()
imagenet_labels = [oi_imagenet_label[lab] for lab in oi_labels]
oi_class = [oi_labels_class[lab] for lab in oi_labels]
imagenet_classes = [oi_imagenet_class[cl] for cl in oi_class]

In [23]:
df_sub.insert(3, "Imagenet_Label", imagenet_labels)
df_sub.insert(4, "Imagenet_Class", imagenet_classes)
# Convert df_sub into a format which CustomImagedataset will accept
# Treat df_sub samples as imagenet samples

df_meta = pd.read_pickle("/bigstor/zsarwar/Imagenet_2012_subsets/MT_DO/DO_1_Imagenet_48_no-wolf-dog-animals.pkl")
cols = df_meta.columns.tolist()

metadata = []
img_names = df_sub['ImageID'].tolist()
for i in range(len(df_sub)):
    t_dict = {}
    img_name = img_names[i] + ".jpg"
    img_path = os.path.join(image_save_path, img_name)
    t_dict['index'] = img_name
    t_dict[cols[0]] = None
    t_dict[cols[1]] = df_sub['Imagenet_Class'].iloc[i]
    t_dict[cols[2]] = df_sub['Imagenet_Label'].iloc[i]
    t_dict[cols[3]] = 'OpenImages'
    t_dict[cols[4]] = 'OpenImages'
    t_dict[cols[5]] = None
    t_dict[cols[6]] = img_path
    metadata.append(t_dict)


df_oi = pd.DataFrame.from_dict(metadata)
df_oi = df_oi.set_index('index')

In [24]:
# Make sure each class has maximum of X samples?
# Drop classes with less than 50 samples
unique_classes = df_oi['class'].unique()
rem_classes = []
for uni_class in unique_classes:
    tot_samples = len(df_oi[df_oi['class'] == uni_class])
    print(uni_class, tot_samples)
    if tot_samples < 25:
        rem_classes.append(uni_class)


dingo, warrigal, warragal, Canis dingo 56
Siberian husky 124
German shepherd, German shepherd dog, German police dog, alsatian 156
beagle 65
Shetland sheepdog, Shetland sheep dog, Shetland 70
Rhodesian ridgeback 15
black-and-tan coonhound 13
Labrador retriever 328
collie 68
Airedale, Airedale terrier 50
basset, basset hound 42
vizsla, Hungarian pointer 34
Lhasa, Lhasa apso 24
standard poodle 40
Doberman, Doberman pinscher 136
Shih-Tzu 321
Border collie 268
Yorkshire terrier 143
cocker spaniel, English cocker spaniel, cocker 369
flat-coated retriever 17
Great Dane 110
German short-haired pointer 230
cairn, cairn terrier 17
Scotch terrier, Scottish terrier, Scottie 20
Tibetan terrier, chrysanthemum dog 17
Italian greyhound 13
curly-coated retriever 11
miniature schnauzer 35
English foxhound 9
boxer 275
miniature poodle 11
Chihuahua 53
toy terrier 29
toy poodle 10
Cardigan, Cardigan Welsh corgi 41
Australian terrier 24
soft-coated wheaten terrier 31
kuvasz 10
whippet 21
affenpinscher, mon

In [25]:
"""
# remove undersampled classes
for cl in rem_classes:
    indices = df_oi[df_oi['class'] == cl].index.tolist()
    df_oi = df_oi.drop(indices)
"""

"\n# remove undersampled classes\nfor cl in rem_classes:\n    indices = df_oi[df_oi['class'] == cl].index.tolist()\n    df_oi = df_oi.drop(indices)\n"

In [26]:
"""
#keep_x = 100
unique_classes = df_oi['label'].unique().tolist()
for uni_class in unique_classes:
    df_uni_class = df_oi[df_oi['label'] == uni_class]
    tot_samples = len(df_uni_class)
    print(tot_samples)
"""

"\n#keep_x = 100\nunique_classes = df_oi['label'].unique().tolist()\nfor uni_class in unique_classes:\n    df_uni_class = df_oi[df_oi['label'] == uni_class]\n    tot_samples = len(df_uni_class)\n    print(tot_samples)\n"

In [27]:
save_path = os.path.join(df_save_path, "df_oi_dogs.pkl")
df_oi.to_pickle(save_path)

In [28]:
df_oi

,image_url,class,label,data_type,dataset,query,img_path
index,,,,,,,
0005474eeb49317d.jpg,None,"dingo, warrigal, warragal, Canis dingo",117,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/0005474eeb49...
000acf666d991c39.jpg,None,Siberian husky,99,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/000acf666d99...
000f24d363c4b66c.jpg,None,"German shepherd, German shepherd dog, German p...",84,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/000f24d363c4...
00100fe58a324932.jpg,None,beagle,11,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/00100fe58a32...
0012d875044bfa35.jpg,None,"Shetland sheepdog, Shetland sheep dog, Shetland",79,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/0012d875044b...
...,...,...,...,...,...,...,...
febe9521e0c5b8e7.jpg,None,Border collie,81,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/febe9521e0c5...
fec6ffc24e220d98.jpg,None,Border collie,81,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/fec6ffc24e22...
ff302a85a54003e0.jpg,None,Border collie,81,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/ff302a85a540...


In [29]:
save_path

'/bigstor/zsarwar/OpenImages/df_oi_dogs.pkl'

In [30]:
df_oi

,image_url,class,label,data_type,dataset,query,img_path
index,,,,,,,
0005474eeb49317d.jpg,None,"dingo, warrigal, warragal, Canis dingo",117,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/0005474eeb49...
000acf666d991c39.jpg,None,Siberian husky,99,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/000acf666d99...
000f24d363c4b66c.jpg,None,"German shepherd, German shepherd dog, German p...",84,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/000f24d363c4...
00100fe58a324932.jpg,None,beagle,11,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/00100fe58a32...
0012d875044bfa35.jpg,None,"Shetland sheepdog, Shetland sheep dog, Shetland",79,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/0012d875044b...
...,...,...,...,...,...,...,...
febe9521e0c5b8e7.jpg,None,Border collie,81,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/febe9521e0c5...
fec6ffc24e220d98.jpg,None,Border collie,81,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/fec6ffc24e22...
ff302a85a54003e0.jpg,None,Border collie,81,OpenImages,OpenImages,None,/bigstor/zsarwar/OpenImages/train/ff302a85a540...
